<a href="https://colab.research.google.com/github/MariaPonomarenko38/NLP_Research_Project/blob/main/paper_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings
warnings.filterwarnings('ignore')

## Install necessary packages and **libraries**

In [2]:
!pip install torch
!pip install transformers
!pip install sentence_transformers
!pip install keras
!pip install tensorflow
!pip install keras_preprocessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 9.2 MB/s 
     |████████████████████████████████| 7.6 MB 45.3 MB/s 
     |████████████████████████████████| 182 kB 67.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 3.3 MB/s 
     |████████████████████████████████| 1.3 MB 16.2 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=3097639d27e09ff8946ba66d2000990f5c145b7911a54e39c21fcd84553ea0b7
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/pub

In [47]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [65]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import nltk
import re
import torch
import uuid
import os 
import csv
import pickle

from collections import Counter

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer


from gensim.models import LdaModel, TfidfModel
from gensim.corpora import Dictionary

from transformers import BertTokenizer, BertModel
from sentence_transformers import SentenceTransformer

from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity

## Read data


In [4]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [5]:
Path = 'drive/My Drive/data'

In [6]:
initial_df = pd.read_csv(Path + '/' + 'projects_proposals.csv')

In [7]:
titles = initial_df["title"]

In [8]:
newdf = initial_df.drop(["supervisor", "company"], axis='columns')

In [9]:
newdf = newdf.reset_index() 
newdf = newdf.drop(['index'], axis=1)

In [10]:
newdf = newdf.drop(['level_0'], axis=1)

In [11]:
newdf = newdf.drop(['Unnamed: 0'], axis=1)

In [12]:
newdf

,title,abstract
0,adapting contextual bandit algorithms to adser...,contextual bandit algorithms are developed to ...
1,defending white box adversarial examples with ...,as deep neural networks become increasingly co...
2,modern gameplay test automation with reinforce...,gameplay testing on computer games to help ens...
3,reducing data loading time in machine learning...,training of models in machine learning is typi...
4,hybrid recommender system for applications bas...,collaborative filtering is a widely used metho...
...,...,...
1113,multilingual semantic similarity of unstructur...,to communicate with their end users businesses...
1114,multilingual semantic search engine using mult...,multiple situations require crosslingual searc...
1115,revealing trends in quantum technology with na...,recent years have witnessed a surge of interes...
1116,adding extra constraints to an open platform f...,acculogic develops stateoftheart testers for s...


In [13]:
titles_docs = pd.Series(newdf['title'])

In [14]:
paper_abstracts_docs = pd.Series(newdf['abstract'])

In [15]:
df_prof = pd.read_excel(Path + '/' + 'professors_data.xlsx')

In [16]:
df_prof 

,Unnamed: 0,professor name,paper_abstract,title,date,Unnamed: 5
0,0,Tarek Abdelrahman,we present a compilation flow for the generati...,A Compilation Flow for the Generation of CNN I...,"8 March, 2022",NaN
1,1,Tarek Abdelrahman,Field experiments were carried on cotton durin...,Toxicological evaluation and residual analysis...,2021-01-01 00:00:00,NaN
2,2,Tarek Abdelrahman,Rice husk was used as an adsorbent for the rem...,Adsorption Study of Cadmium (II) and Lead (II)...,2020-08-01 00:00:00,NaN
3,3,Tarek Abdelrahman,the fungicidal residues of certain copper fung...,Residual effect of certain copper fungicides u...,2017-02-10 00:00:00,NaN
4,4,Tarek Abdelrahman,This study explored the heavy metals contamina...,Monitoring and minimizing levels of some heavy...,2017,NaN
...,...,...,...,...,...,...
1446,1446,Andrea Tagliasacchi,when representing a solid object there are alt...,Skeletal Representations and Applications,"25 June, 2014",NaN
1447,1447,Adrian Butscher,we present a method for enforcing manufacturab...,A subtractive manufacturing constraint for lev...,"19 February, 2020",NaN
1448,1448,Adrian Butscher,we present a generalization of the bilateral f...,A General Framework for Bilateral and Mean Shi...,"30 April, 2014",NaN
1449,1449,Adrian Butscher,the gluing technique is used to construct hype...,A Gluing Construction for Prescribed Mean Curv...,"19 February, 2009",NaN


In [36]:
only_abstract_title = df_prof[['paper_abstract', 'title']]
only_abstract_title = only_abstract_title.rename(columns={"paper_abstract": "abstract"})
only_abstract_title=only_abstract_title.reindex(columns=['title', 'abstract'])

In [37]:
only_abstract_title

,title,abstract
0,A Compilation Flow for the Generation of CNN I...,we present a compilation flow for the generati...
1,Toxicological evaluation and residual analysis...,Field experiments were carried on cotton durin...
2,Adsorption Study of Cadmium (II) and Lead (II)...,Rice husk was used as an adsorbent for the rem...
3,Residual effect of certain copper fungicides u...,the fungicidal residues of certain copper fung...
4,Monitoring and minimizing levels of some heavy...,This study explored the heavy metals contamina...
...,...,...
1446,Skeletal Representations and Applications,when representing a solid object there are alt...
1447,A subtractive manufacturing constraint for lev...,we present a method for enforcing manufacturab...
1448,A General Framework for Bilateral and Mean Shi...,we present a generalization of the bilateral f...
1449,A Gluing Construction for Prescribed Mean Curv...,the gluing technique is used to construct hype...


In [42]:
projects_and_papers = pd.concat([only_abstract_title, newdf], ignore_index=True)

In [17]:
df_prof = df_prof.drop(['Unnamed: 0','Unnamed: 5'], axis=1)

In [22]:
professors = pd.Series(df_prof['professor name'])

In [23]:
abstracts = pd.Series(df_prof['paper_abstract'])
abstracts_const = pd.Series(df_prof['paper_abstract'])
for i in range(len(abstracts)):
    abstracts[i] = abstracts[i] + titles[i]

## Data Preprocessing

In [24]:
def remove_punctuation_and_digits_df(df):
    tokenizer = RegexpTokenizer(r'\w+')
    for index, row in df.iterrows():
        preprocessed_string = " ".join(tokenizer.tokenize(row['abstract'])) # remove punctuaion 
        preprocessed_string_without_digits = re.sub(r'\d+', '', preprocessed_string) # remove digits 
        preprocessed_string_without_digits_lowercase = preprocessed_string_without_digits.lower()
        df.at[index,'abstract'] = preprocessed_string_without_digits_lowercase
    return df

In [43]:
preprocessed_df = remove_punctuation_and_digits_df(projects_and_papers)

In [45]:
def remove_stop_words_df(df):
    stop_words = set(stopwords.words('english'))
    for index, row in df.iterrows():
        word_tokens = word_tokenize(row['abstract'])
        filtered_sentence = []
        for w in word_tokens:
            if w not in stop_words:
                filtered_sentence.append(w)
        df.at[index,'abstract'] = " ".join(filtered_sentence)
    return df

In [48]:
preprocessed_stop_words_df = remove_stop_words_df(projects_and_papers)

In [50]:
abstracts = preprocessed_stop_words_df['abstract']

In [63]:
titles = preprocessed_stop_words_df['title']

In [52]:
abstracts_tokenized = abstracts.apply(lambda x: x.split())

## Exploring different extraction technique for text similarity tasks

### BERT

In [53]:
import torch
from transformers import BertTokenizer, BertModel

bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [54]:
prebert_tok_docs = abstracts.apply(lambda x: "[CLS] " + x + " [SEP]")

In [56]:
bert_docs  = prebert_tok_docs.apply(lambda x: bert_tokenizer.tokenize(x))

In [58]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
embeddings = model.encode(bert_docs, show_progress_bar=True)

Batches:   0%|          | 0/81 [00:00<?, ?it/s]

In [61]:
project_start_ind = 1451

In [62]:
def find_sim_bert(ind):
    cosineSimilarities = []
    for i in range(project_start_ind):
        cosineSimilarities.append((cosine_similarity([embeddings[ind]],  [embeddings[i]]).flatten(), i))
    return sorted(cosineSimilarities, reverse=True)[:3]

In [76]:
predict_paper_bert = pd.DataFrame(columns=['Professor','Paper title', 'Paper abstract', 'Proposal title', 'Proposal abstract' ])
li = []
for i in range(project_start_ind, project_start_ind + 10):
    result = find_sim_bert(i)
    for j in range(len(result)):
        proj_ind = result[j][1]
        predict_paper_bert  = predict_paper_bert .append({'Professor': professors[proj_ind], 'Paper title': titles[proj_ind],  'Paper abstract':abstracts[proj_ind],
                         'Proposal title': titles[i], 'Proposal abstract':abstracts[i]},ignore_index=True)

In [77]:
predict_paper_bert

,Professor,Paper title,Paper abstract,Proposal title,Proposal abstract
0,Kelly Lyons,How ReadMe files are structured in open source...,context recent studies open source platforms g...,adapting contextual bandit algorithms to adser...,contextual bandit algorithms developed gain ma...
1,Yang Xu,HistBERT: A Pre-trained Language Model for Dia...,contextualized word embeddings demonstrated st...,adapting contextual bandit algorithms to adser...,contextual bandit algorithms developed gain ma...
2,Sven Dickinson,A Framework for Symmetric Part Detection in Cl...,role symmetry computer vision waxed waned impo...,adapting contextual bandit algorithms to adser...,contextual bandit algorithms developed gain ma...
3,Garth Gibson,Hardware Sensitivity Analysis for Deep Learnin...,deep learning revolutionized machine learning ...,defending white box adversarial examples with ...,deep neural networks become increasingly commo...
4,Alan Moses,Systematic identification of conditionally fol...,deep learning based approaches protein structu...,defending white box adversarial examples with ...,deep neural networks become increasingly commo...
5,Quaid Morris,DeepTumour: Identify tumor origin from whole g...,deeptumour algorithm predicts tissue origin tu...,defending white box adversarial examples with ...,deep neural networks become increasingly commo...
6,Ken Jackson,Backward simulation of multivariate mixed Pois...,simulation correlated multivariate poisson pro...,modern gameplay test automation with reinforce...,gameplay testing computer games help ensure hi...
7,Igor Gilitschenski,"VISTA 2.0: An Open, Data-driven Simulator for ...",simulation potential transform development rob...,modern gameplay test automation with reinforce...,gameplay testing computer games help ensure hi...
8,Alex Mihailidis,Unsupervised deep learning to detect agitation...,behavioural symptoms dementia present signific...,modern gameplay test automation with reinforce...,gameplay testing computer games help ensure hi...
9,Garth Gibson,Primitives for Dynamic Big Model Parallelism,training large machine learning models many va...,reducing data loading time in machine learning...,training models machine learning typically don...


###TF-IDF

In [72]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
tf_idf = vectorizer.fit_transform(abstracts)

def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]

In [73]:
from sklearn.metrics.pairwise import cosine_similarity

def find_sim_tf_idf(query):
    query_tfidf = vectorizer.transform([query])
    cosineSimilarities = cosine_similarity(query_tfidf,  tf_idf[:project_start_ind]).flatten()
    cosineSimilarities = [(cosineSimilarities[i], i) for i in range(len(cosineSimilarities))]
    return sorted(cosineSimilarities, reverse=True)[:3]

In [83]:
predict_paper_tf_idf = pd.DataFrame(columns=['Proposal title', 'Proposal abstract', 'Paper title',  'Paper abstract',  'Professor'])
for i in range(project_start_ind, project_start_ind + 10):
    res1 = find_sim_tf_idf(abstracts[i])
    for res in res1: 
        similarity = res[0]
        paper_ind = res[1]
        dicti = {'Paper title': titles[paper_ind], 'Proposal title': titles[i], 'Professor': professors[paper_ind],'Proposal abstract': abstracts[i], 'Paper abstract': abstracts[paper_ind]}
        predict_paper_tf_idf = predict_paper_tf_idf.append(dicti, ignore_index=True)

In [84]:
predict_paper_tf_idf

,Proposal title,Proposal abstract,Paper title,Paper abstract,Professor
0,adapting contextual bandit algorithms to adser...,contextual bandit algorithms developed gain ma...,Boosting for Online Convex Optimization,consider decisionmaking framework online conve...,Karan Singh
1,adapting contextual bandit algorithms to adser...,contextual bandit algorithms developed gain ma...,An Experimental Study of Algorithms for Online...,perform experimental study algorithms online b...,Allan Borodin
2,adapting contextual bandit algorithms to adser...,contextual bandit algorithms developed gain ma...,Fair algorithms for multi-agent multi-armed ba...,propose multi agent variant classical multi ar...,Nisarg Shah
3,defending white box adversarial examples with ...,deep neural networks become increasingly commo...,Efficient Adversarial Training With Data Pruning,neural networks susceptible adversarial exampl...,Nicolas Papernot
4,defending white box adversarial examples with ...,deep neural networks become increasingly commo...,Stochastic Combinatorial Ensembles for Defendi...,many deep learning algorithms easily fooled si...,Benjamin Haibe-Kains
5,defending white box adversarial examples with ...,deep neural networks become increasingly commo...,Stochastic Combinatorial Ensembles for Defendi...,many deep learning algorithms easily fooled si...,David Duvenaud
6,modern gameplay test automation with reinforce...,gameplay testing computer games help ensure hi...,RL-Scope: Cross-stack Profiling for Deep Reinf...,deep reinforcement learning rl made groundbrea...,Moshe Gabel
7,modern gameplay test automation with reinforce...,gameplay testing computer games help ensure hi...,Using a serious game to measure executive func...,previous research developed serious target acq...,Mark Chignell
8,modern gameplay test automation with reinforce...,gameplay testing computer games help ensure hi...,RL-Scope: Cross-Stack Profiling for Deep Reinf...,deep reinforcement learning rl made groundbrea...,Moshe Gabel
9,reducing data loading time in machine learning...,training models machine learning typically don...,Allocation of graphics processing units for vi...,present disclosure provides systems methods co...,Eyal de Lara


### <h3> Word2Vec </h3>

In [85]:
max_len = 0
for i in abstracts_tokenized:
    if len(i) > max_len:
        max_len = len(i)

In [86]:
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

tokenizer=Tokenizer()
tokenizer.fit_on_texts(abstracts_tokenized)
tokenized_documents=tokenizer.texts_to_sequences(abstracts_tokenized)
tokenized_paded_documents=pad_sequences(tokenized_documents,maxlen=max_len,padding='post')
vocab_size = len(tokenizer.word_index) + 1
print(tokenized_paded_documents[0])

[   58  7398   694   234  1363   185  3072  5554   694  9006  4215     3
  7399  1469  5555  2383   242  7400  7399  3676  3925  5556 12027    53
   111   330   340  2919   268  1000  7399  1469   330  5555  2919   228
  2197   246   135   253  1113   228  5008  2284  2573   235  1271  2919
  5555   166   695  3072 12028 12029  3251  7400 12030    19  2919    53
    11   330  3072   470  1000  3072    11   809  3072   470    72  2481
  2574   470    72 12031  5555   470   182  5555  1880   809  1469    16
  1648  1041   330   271    22    16   242  2920   728   318  1364  1365
    34 12032    22  2921 12033  1881   195   681    22   471 12034   454
   303   784    11   417  4579  5009   462  5554   139   340    40  1599
  1001  7398   694   234  1363   185  3072  5554     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0   

In [87]:
from gensim.models import Word2Vec as w2v

model_w2v = w2v(
    abstracts_tokenized
)       

embedding_matrix=np.zeros((vocab_size,100))
for word,i in tokenizer.word_index.items():
    if word in model_w2v:
        embedding_matrix[i]=model_w2v[word]

document_word_embeddings=np.zeros((len(tokenized_paded_documents),max_len,100))
for i in range(len(tokenized_paded_documents)):
    for j in range(len(tokenized_paded_documents[0])):
        document_word_embeddings[i][j]=embedding_matrix[tokenized_paded_documents[i][j]]
document_word_embeddings.shape

(2569, 232, 100)

In [88]:
document_embeddings=np.zeros((len(tokenized_paded_documents),100))
words=vectorizer.get_feature_names()
for i in range(len(document_word_embeddings)):
    for j in range(len(document_word_embeddings[i])):
        document_embeddings[i] += document_word_embeddings[i][j]#*tfidf_vectors[i][j]
    document_embeddings[i] = document_embeddings[i] / len(document_word_embeddings)

In [99]:
def find_sim_word2vec(ind):
    cosineSimilarities = []
    for i in range(project_start_ind):
        cosineSimilarities.append((cosine_similarity([document_embeddings[ind]],  [document_embeddings[i]])[0][0], i))
    return sorted(cosineSimilarities, reverse=True)[:3]

In [100]:
find_sim_word2vec(1452)

[(0.9998521713487564, 430),
 (0.9998402137485728, 439),
 (0.9998030954020095, 479)]

In [110]:
predict_paper_word2vec = pd.DataFrame(columns=['Proposal title', 'Proposal abstract', 'Paper title',  'Paper abstract',  'Professor'])
for i in range(project_start_ind, project_start_ind + 10):
    res1 = find_sim_word2vec(i)
    for res in res1: 
        paper_ind = res[1]
        dicti = {'Proposal title': titles[i], 'Paper abstract': abstracts[paper_ind], 'Paper title': titles[paper_ind], 'Proposal abstract': abstracts[i], 'Professor': professors[paper_ind],}
        predict_paper_word2vec = predict_paper_word2vec.append(dicti, ignore_index=True)

In [111]:
predict_paper_word2vec

,Proposal title,Proposal abstract,Paper title,Paper abstract,Professor
0,adapting contextual bandit algorithms to adser...,contextual bandit algorithms developed gain ma...,Transparent Anonymization: Thwarting Adversari...,numerous generalization techniques proposed pr...,Nick Koudas
1,adapting contextual bandit algorithms to adser...,contextual bandit algorithms developed gain ma...,Continuous-Time Fitted Value Iteration for Rob...,solving hamiltonjacobibellman equation importa...,Animesh Garg
2,adapting contextual bandit algorithms to adser...,contextual bandit algorithms developed gain ma...,Causal Modeling for Fairness in Dynamical Systems,many application areaslending education online...,Richard Zemel
3,defending white box adversarial examples with ...,deep neural networks become increasingly commo...,AutoInt: Automatic Integration for Fast Neural...,numerical integration foundational technique s...,David Lindell
4,defending white box adversarial examples with ...,deep neural networks become increasingly commo...,Autoint: Automatic integration for fast neural...,numerical integration foundational technique s...,David Lindell
5,defending white box adversarial examples with ...,deep neural networks become increasingly commo...,Microscopic generative models for complex netw...,last decade growing interest understanding com...,Peter Marbach
6,modern gameplay test automation with reinforce...,gameplay testing computer games help ensure hi...,Pisces: Efficient Federated Learning via Guide...,federated learning fl typically performed sync...,Baochun Li
7,modern gameplay test automation with reinforce...,gameplay testing computer games help ensure hi...,Learning to Give Checkable Answers with Prover...,ability know trust decisions made machine lear...,Roger Grosse
8,modern gameplay test automation with reinforce...,gameplay testing computer games help ensure hi...,Fed-Sim: Federated Simulation for Medical Imaging,labelling data expensive time consuming especi...,Sanja Fidler
9,reducing data loading time in machine learning...,training models machine learning typically don...,Dynamic Measurement Scheduling for Event Forec...,imagine patient critical condition measured fo...,Anna Goldenberg


###<h3> Doc2Vec Embeddings </h3>

In [112]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(abstracts_tokenized)]

In [113]:
model = Doc2Vec(tagged_data[:1451], vector_size = 700, window = 2, min_count = 1, epochs = 500)

'\nvector_size = Dimensionality of the feature vectors.\nwindow = The maximum distance between the current and predicted word within a sentence.\nmin_count = Ignores all words with total frequency lower than this.\nalpha = The initial learning rate.\n'

In [114]:
def find_sim_doc2vec(ind):
    result = 0
    test_doc_vector = model.infer_vector(abstracts_tokenized[ind])
    result = model.docvecs.most_similar(positive = [test_doc_vector])
    return result[:3]

In [115]:
predict_paper_doc2vec = pd.DataFrame(columns=['Proposal title', 'Proposal abstract', 'Paper title',  'Paper abstract',  'Professor'])
for i in range(project_start_ind, project_start_ind + 10):
    res1 = find_sim_doc2vec(i)
    for res in res1: 
        paper_ind = res[0]
        dicti = {'Proposal title': titles[i], 'Paper abstract': abstracts[paper_ind], 'Paper title': titles[paper_ind], 'Proposal abstract': abstracts[i], 'Professor': professors[paper_ind],}
        predict_paper_doc2vec = predict_paper_doc2vec.append(dicti, ignore_index=True)

In [116]:
predict_paper_doc2vec

,Proposal title,Proposal abstract,Paper title,Paper abstract,Professor
0,adapting contextual bandit algorithms to adser...,contextual bandit algorithms developed gain ma...,Fair algorithms for multi-agent multi-armed ba...,propose multi agent variant classical multi ar...,Nisarg Shah
1,adapting contextual bandit algorithms to adser...,contextual bandit algorithms developed gain ma...,RL-Scope: Cross-Stack Profiling for Deep Reinf...,deep reinforcement learning rl made groundbrea...,Moshe Gabel
2,adapting contextual bandit algorithms to adser...,contextual bandit algorithms developed gain ma...,RL-Scope: Cross-Stack Profiling for Deep Reinf...,deep reinforcement learning rl made groundbrea...,Gennady Pekhimenko
3,defending white box adversarial examples with ...,deep neural networks become increasingly commo...,Manipulating sgd with data ordering attacks,machine learning vulnerable wide variety attac...,Murat Erdogdu
4,defending white box adversarial examples with ...,deep neural networks become increasingly commo...,Dataset Inference: Ownership Resolution in Mac...,increasingly data computation involved trainin...,Nicolas Papernot
5,defending white box adversarial examples with ...,deep neural networks become increasingly commo...,Towards Optimal Strategies for Training Self-D...,autonomous driving relies huge volume realworl...,Sanja Fidler
6,modern gameplay test automation with reinforce...,gameplay testing computer games help ensure hi...,Automating Clinical Documentation with Digital...,recently digital scribe systems gaining popula...,Khai Truong
7,modern gameplay test automation with reinforce...,gameplay testing computer games help ensure hi...,Low-level Voice and Hand-Tracking Interaction ...,hand tracking allows users engage virtual envi...,Cosmin Munteanu
8,modern gameplay test automation with reinforce...,gameplay testing computer games help ensure hi...,Improving domain adaptation in de-identificati...,de identification fundamental task electronic ...,Zhaolei Zhang
9,reducing data loading time in machine learning...,training models machine learning typically don...,Large scale distributed neural network trainin...,techniques ensembling distillation promise mod...,Geoffrey Hinton
